In [6]:
import os
import mindspore as ms
# 导入mindspore中context模块，用于配置当前执行环境，包括执行模式等特性。
import mindspore.context as context
# c_transforms模块提供常用操作，包括OneHotOp和TypeCast
import mindspore.dataset.transforms as C
# vision.c_transforms模块是处理图像增强的高性能模块，用于数据增强图像数据改进训练模型。
import mindspore.dataset.vision as CV
import numpy as np
from mindspore import nn
from mindspore.nn import Accuracy
from mindspore.train import Model
from mindspore.train.callback import LossMonitor
import matplotlib.pyplot as plt
# 设置MindSpore的执行模式和设备
context.set_context(mode=context.GRAPH_MODE, device_target='CPU') # Ascend, CPU, GPU
 
def create_dataset(data_dir, training=True, batch_size=32, resize=(32, 32),
                   rescale=1/(255*0.3081), shift=-0.1307/0.3081, buffer_size=64):
    data_train = os.path.join(data_dir, 'train') # 训练集信息
    data_test = os.path.join(data_dir, 'test') # 测试集信息
    ds = ms.dataset.MnistDataset(data_train if training else data_test)
    ds = ds.map(input_columns=["image"], operations=[CV.Resize(resize), CV.Rescale(rescale, shift), CV.HWC2CHW()])
    ds = ds.map(input_columns=["label"], operations=C.TypeCast(ms.int32))
    ds = ds.shuffle(buffer_size=buffer_size).batch(batch_size, drop_remainder=True)
    return ds
 
ds = create_dataset(r'C:\Users\19616\MNIST_Data', training=False)
# data = ds.create_dict_iterator()
# for da in data:
#     print(da['image'].type())
# images = data['image'].asnumpy()
# labels = data['label'].asnumpy()
# #显示前4张图片以及对应标签
# for i in range(1, 5):
#     plt.subplot(2, 2, i)
#     plt.imshow(images[i][0])
#     plt.title('Number: %s' % labels[i])
#     plt.xticks([])
# plt.show()
 
#定义LeNet5模型
class LeNet5(nn.Cell):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Dense(16*5*5, 120)
        self.fc2 = nn.Dense(120, 84)
        self.fc3 = nn.Dense(84, 10)

    def construct(self, x):
        x = self.relu(self.conv1(x))
        x = self.max_pool2d(x)
        x = self.relu(self.conv2(x))
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
 
def train(data_dir, lr=0.01, momentum=0.9, num_epochs=10):
    ds_train = create_dataset(data_dir)
    ds_eval = create_dataset(data_dir, training=False)
    net = LeNet5()
    #计算softmax交叉熵。
    loss = nn.loss.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    #设置Momentum优化器
    opt = nn.Momentum(net.trainable_params(), lr, momentum)
    
    loss_cb = LossMonitor(per_print_times=ds_train.get_dataset_size())
    
    model = Model(net, loss, opt, metrics={"Accuracy": Accuracy()})
    
    model.train(num_epochs, ds_train, callbacks=[loss_cb], dataset_sink_mode=True)

    
    metrics_result = model.eval(ds_eval)
    
    print('Accuracy:',metrics_result["Accuracy"])
    
    






 
train(r'C:\Users\19616\MNIST_Data')




epoch: 1 step: 1875, loss is 0.05679894983768463
epoch: 2 step: 1875, loss is 0.10386011004447937
epoch: 3 step: 1875, loss is 0.005582510028034449
epoch: 4 step: 1875, loss is 0.0019453082932159305
epoch: 5 step: 1875, loss is 0.0004732749657705426
epoch: 6 step: 1875, loss is 0.0005306907114572823
epoch: 7 step: 1875, loss is 0.07282890379428864
epoch: 8 step: 1875, loss is 0.00039607385406270623
epoch: 9 step: 1875, loss is 3.834067319985479e-05
epoch: 10 step: 1875, loss is 0.00044187414459884167
Accuracy: 0.9896834935897436
